In [ ]:
import numpy as np
from pathlib import Path
import utils
import mne


In [ ]:
data_preprocessed = mne.io.read_raw_edf(Path('preprocessed_data/chb21/chb21_19.edf'), preload=False, verbose=False)


In [ ]:
start_ev = np.array([1288,2000])
end_ev = np.array([1344,2100])
input_arr = data_preprocessed.get_data(stop=10000)
interictal_period = (
            np.expand_dims(input_arr.transpose(), axis=2)
            .swapaxes(0, 2)
            .swapaxes(0, 1)
        ) 

In [ ]:
interictal_features, interictal_event_labels, interictal_event_time_labels,seizure_features,seizure_event_labels = utils.extract_training_data_and_labels(data_preprocessed,start_ev,end_ev,overlap=0)

In [ ]:
seizure_features.shape

In [ ]:
%matplotlib qt

In [ ]:
adult_data =Path('raw_dataset')
peprocessed_data = Path('preprocessed_data')
subject_seizures = Path('raw_dataset/RECORDS-WITH-SEIZURES')

In [ ]:
utils.preprocess_dataset(subject_seizures,adult_data,peprocessed_data)

In [ ]:
data_raw =utils.load_and_dump_channels(Path('raw_dataset/chb01/chb01_03.edf'))
utils.reorder_channels_chbmit(data_raw)
raw_copy = utils.run_preprocessing(data_raw,18,apply_ica=False,apply_pca = True,freq_l=2)